## 1. Setup de ambiente (Python + Jupyter)

In [1]:
# ==========================================
# Vista analítica fact_join (fact + dimensões)
# ==========================================

# 1) junta FACT_VENDA com DIM_TEMPO
fact_join = fact_venda.merge(
    dim_tempo[["id_tempo", "data", "ano", "mes", "trimestre"]],
    on="id_tempo",
    how="left"
)

# 2) junta DIM_CLIENTE
fact_join = fact_join.merge(
    dim_cliente[["id_cliente", "nome_cliente", "segmento_cliente", "cidade", "pais"]],
    on="id_cliente",
    how="left"
)

# 3) junta DIM_ARTIGO
fact_join = fact_join.merge(
    dim_artigo[["id_artigo", "nome_artigo", "categoria", "subcategoria"]],
    on="id_artigo",
    how="left"
)

# 4) junta DIM_VENDEDOR
fact_join = fact_join.merge(
    dim_vendedor[["id_vendedor", "nome_vendedor", "equipa"]],
    on="id_vendedor",
    how="left"
)

# 5) inspeção rápida
fact_join.head()

NameError: name 'fact_venda' is not defined

# PHC — Mock Analytics Notebook

Notebook técnico para simular um **pipeline de dados analítico** baseado num ERP (estilo PHC), usando **dados mock**.

## Objetivos
- Modelar dados em **Star Schema** (Fact + Dimensões)
- Consolidar dados numa **fact table de vendas**
- Calcular **KPIs de negócio**
- Exportar datasets prontos para **Power BI / Metabase**

In [ ]:
import pandas as pd
import numpy as np

# Mostrar mais colunas nas tabelas
pd.set_option("display.max_columns", 50)

print("Setup OK")


## 2. Modelo de dados mock (estilo PHC)

Aqui definimos 4 tabelas principais que imitam a estrutura habitual de um ERP:

1. **CLIENTES** – quem compra.
2. **ARTIGOS** – o que é vendido.
3. **DOCUMENTOS DE VENDA** – faturas / documentos comerciais.
4. **LINHAS DE VENDA** – detalhe de cada documento.

Mais tarde estas tabelas seriam substituídas pelas tabelas reais da BD PHC.

### 2.1 Clientes

In [ ]:
# ============================
# Tabela CLIENTES (mock PHC)
# ============================

clientes = pd.DataFrame({
    "cliente_id": [1, 2, 3, 4, 5],
    "nome": ["Cliente A", "Cliente B", "Cliente C", "Cliente D", "Cliente E"],
    "nif": ["123456789", "987654321", "192837465", "564738291", "102938475"],
    "segmento": ["Retalho", "Serviços", "Retalho", "Industrial", "Serviços"],
    "cidade": ["Lisboa", "Porto", "Braga", "Coimbra", "Lisboa"]
})

clientes

### 2.2 Artigos

In [ ]:
# ============================
# Tabela ARTIGOS (mock PHC)
# ============================

artigos = pd.DataFrame({
    "artigo_id": [101, 102, 103, 104],
    "descricao": ["Teclado", "Rato", "Monitor 24", "Portátil"],
    "preco_venda": [15.90, 9.99, 129.90, 799.00],
    "categoria": ["Periféricos", "Periféricos", "Monitores", "Computadores"]
})

artigos

### 2.3 Documentos de venda

In [ ]:
# ============================
# Tabela DOCUMENTOS DE VENDA (mock PHC)
# ============================

documentos = pd.DataFrame({
    "doc_id": [1001, 1002, 1003, 1004, 1005],
    "cliente_id": [1, 2, 3, 1, 5],    # liga ao cliente
    "data": pd.to_datetime([
        "2025-01-10",
        "2025-01-12",
        "2025-01-15",
        "2025-02-02",
        "2025-02-10"
    ]),
    "tipo_doc": ["FT", "FT", "FT", "FT", "FT"]  # Fatura
})

documentos

### 2.4 Linhas de venda

In [ ]:
# ============================
# Tabela LINHAS DE VENDA (mock PHC)
# ============================

linhas = pd.DataFrame({
    "linha_id": [1, 2, 3, 4, 5, 6, 7],
    "doc_id": [1001, 1001, 1002, 1003, 1004, 1005, 1005],  # faturas
    "artigo_id": [101, 102, 103, 101, 104, 102, 103],       # produtos vendidos
    "quantidade": [2, 1, 1, 3, 1, 2, 1],
})

# Calcular valor total da linha
linhas["valor_total"] = linhas.apply(
    lambda row: row["quantidade"] * artigos.loc[artigos["artigo_id"] == row["artigo_id"], "preco_venda"].values[0],
    axis=1
)

linhas

## 3. Consolidação das tabelas (JOINs)

Aqui simulamos o que irá acontecer na BD PHC real:

- `linhas` ⨝ `artigos`  → adiciona preço e descrição a cada linha.
- `documentos` ⨝ `clientes` → adiciona dados do cliente ao documento.
- Resultado final: tabela **vendas** com toda a informação necessária para KPIs e dashboards.

In [ ]:
# ============================
# Consolidação de vendas (JOIN)
# ============================

# Juntar linhas → artigos (para obter o preço e descrição)
linhas_artigos = linhas.merge(artigos, on="artigo_id", how="left")

# Juntar documentos → clientes
docs_clientes = documentos.merge(clientes, on="cliente_id", how="left")

# Juntar tudo
vendas = linhas_artigos.merge(docs_clientes, on="doc_id", how="left")

vendas

## 4. KPIs base

KPIs calculados a partir da tabela `vendas`:

1. **Total de faturação**.
2. **Top clientes por faturação**.
3. **Top artigos por faturação**.
4. **Faturação mensal** (YYYY-MM).

Estes KPIs podem ser replicados depois em SQL ou diretamente na ferramenta de BI.

In [ ]:
# KPI 1 — Total de faturação
kpi_total_faturacao = vendas["valor_total"].sum()
kpi_total_faturacao

In [ ]:
# KPI 2 — Top clientes por faturação
kpi_top_clientes = (
    vendas.groupby("nome")["valor_total"]
    .sum()
    .sort_values(ascending=False)
)
kpi_top_clientes

In [ ]:
# KPI 3 — Top artigos por faturação
kpi_top_artigos = (
    vendas.groupby("descricao")["valor_total"]
    .sum()
    .sort_values(ascending=False)
)
kpi_top_artigos

In [ ]:
# KPI 4 — Vendas por mês (YYYY-MM)
vendas["ano_mes"] = vendas["data"].dt.to_period("M").astype(str)

kpi_vendas_mensais = (
    vendas.groupby("ano_mes")["valor_total"]
    .sum()
    .sort_values()
)
kpi_vendas_mensais

## 5. Visualização tipo "mini-dashboard"

Aqui criamos um layout simples em Matplotlib que junta:

- Top clientes por faturação.
- Top artigos por faturação.
- Faturação mensal.

É apenas um **rascunho visual** para ajudar a desenhar o dashboard final em Power BI / Metabase.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Criar layout com 2 linhas e 2 colunas
fig = plt.figure(figsize=(12, 8))
gs = GridSpec(2, 2, figure=fig)

# Gráfico 1 — Faturação por Cliente
ax1 = fig.add_subplot(gs[0, 0])
ax1.bar(kpi_top_clientes.index, kpi_top_clientes.values)
ax1.set_title("Top Clientes por Faturação")
ax1.set_ylabel("€")
ax1.tick_params(axis='x', rotation=45)

# Gráfico 2 — Top Artigos
ax2 = fig.add_subplot(gs[0, 1])
ax2.bar(kpi_top_artigos.index, kpi_top_artigos.values, color="orange")
ax2.set_title("Top Artigos por Faturação")
ax2.set_ylabel("€")
ax2.tick_params(axis='x', rotation=45)

# Gráfico 3 — Faturação Mensal (linha)
ax3 = fig.add_subplot(gs[1, :])
ax3.plot(kpi_vendas_mensais.index, kpi_vendas_mensais.values, marker="o")
ax3.set_title("Faturação Mensal")
ax3.set_ylabel("€")
ax3.set_xlabel("Mês")
ax3.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)  # reprodutibilidade

# -------------------------
# DIM_CLIENTE
# -------------------------
n_clientes = 50
dim_cliente = pd.DataFrame({
    "id_cliente": range(1, n_clientes + 1),
    "nome_cliente": [f"Cliente {i}" for i in range(1, n_clientes + 1)],
    "segmento_cliente": np.random.choice(["Retail", "B2B", "Parceiro"], size=n_clientes),
    "cidade": np.random.choice(["Porto", "Lisboa", "Braga", "Coimbra"], size=n_clientes),
    "pais": "Portugal"
})

# -------------------------
# DIM_ARTIGO
# -------------------------
n_artigos = 30
dim_artigo = pd.DataFrame({
    "id_artigo": range(1, n_artigos + 1),
    "nome_artigo": [f"Artigo {i}" for i in range(1, n_artigos + 1)],
    "categoria": np.random.choice(["Software", "Serviço", "Hardware"], size=n_artigos),
    "subcategoria": np.random.choice(["Licença", "Implementação", "Suporte", "Equipamento"], size=n_artigos)
})

# -------------------------
# DIM_VENDEDOR
# -------------------------
n_vendedores = 8
dim_vendedor = pd.DataFrame({
    "id_vendedor": range(1, n_vendedores + 1),
    "nome_vendedor": [f"Vendedor {i}" for i in range(1, n_vendedores + 1)],
    "equipa": np.random.choice(["Norte", "Sul", "Online"], size=n_vendedores)
})

# -------------------------
# DIM_TEMPO
# -------------------------
datas = pd.date_range(start="2024-01-01", end="2024-12-31", freq="D")
dim_tempo = pd.DataFrame({
    "id_tempo": range(1, len(datas) + 1),
    "data": datas,
})
dim_tempo["ano"] = dim_tempo["data"].dt.year
dim_tempo["mes"] = dim_tempo["data"].dt.month
dim_tempo["trimestre"] = dim_tempo["data"].dt.quarter
dim_tempo["dia_semana"] = dim_tempo["data"].dt.day_name(locale="pt_PT")

# -------------------------
# FACT_VENDA
# -------------------------
n_vendas = 3000  # ajusta se quiseres mais/menos registos

fact_venda = pd.DataFrame({
    "id_venda": range(1, n_vendas + 1),
    "id_tempo": np.random.choice(dim_tempo["id_tempo"], size=n_vendas),
    "id_cliente": np.random.choice(dim_cliente["id_cliente"], size=n_vendas),
    "id_artigo": np.random.choice(dim_artigo["id_artigo"], size=n_vendas),
    "id_vendedor": np.random.choice(dim_vendedor["id_vendedor"], size=n_vendas),
    "quantidade": np.random.randint(1, 10, size=n_vendas),
    "preco_unitario": np.random.uniform(50, 1000, size=n_vendas).round(2),
    "desconto_perc": np.random.choice([0, 5, 10, 15, 20], size=n_vendas)
})

# calcula totais
fact_venda["custo_unitario"] = (fact_venda["preco_unitario"] * np.random.uniform(0.4, 0.8, size=n_vendas)).round(2)
fact_venda["total_bruto"] = (fact_venda["quantidade"] * fact_venda["preco_unitario"]).round(2)
fact_venda["total_desconto"] = (fact_venda["total_bruto"] * fact_venda["desconto_perc"] / 100).round(2)
fact_venda["total_liquido"] = (fact_venda["total_bruto"] - fact_venda["total_desconto"]).round(2)
fact_venda["custo_total"] = (fact_venda["quantidade"] * fact_venda["custo_unitario"]).round(2)
fact_venda["margem"] = (fact_venda["total_liquido"] - fact_venda["custo_total"]).round(2)

# quick check
dim_cliente.head(), dim_artigo.head(), dim_vendedor.head(), dim_tempo.head(), fact_venda.head()

In [ ]:
dim_cliente.to_csv("dim_cliente.csv", index=False)
dim_artigo.to_csv("dim_artigo.csv", index=False)
dim_vendedor.to_csv("dim_vendedor.csv", index=False)
dim_tempo.to_csv("dim_tempo.csv", index=False)
fact_venda.to_csv("fact_venda.csv", index=False)

In [ ]:
# ==============================
# KPIs principais da dashboard
# ==============================

faturacao_total = fact_join["total_liquido"].sum()
margem_total = fact_join["margem"].sum()
n_vendas = fact_join["id_venda"].nunique()
n_clientes = dim_cliente["id_cliente"].nunique()
n_artigos = dim_artigo["id_artigo"].nunique()

ticket_medio = faturacao_total / n_vendas
faturacao_media_por_cliente = faturacao_total / n_clientes
margem_pct = (margem_total / faturacao_total) * 100

kpi_cards = pd.DataFrame({
    "KPI": [
        "Faturação total (€)",
        "Margem total (€)",
        "Margem (%)",
        "Nº vendas",
        "Nº clientes",
        "Nº artigos",
        "Ticket médio (€)",
        "Faturação média por cliente (€)",
    ],
    "Valor": [
        round(faturacao_total, 2),
        round(margem_total, 2),
        round(margem_pct, 2),
        int(n_vendas),
        int(n_clientes),
        int(n_artigos),
        round(ticket_medio, 2),
        round(faturacao_media_por_cliente, 2),
    ],
})

kpi_cards

In [ ]:
# DASHBOARD PHC - PIPELINE LIMPO (MOCK STAR SCHEMA)

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)  # reprodutibilidade

# -------------------------
# DIM_CLIENTE
# -------------------------
n_clientes = 50
dim_cliente = pd.DataFrame({
    "id_cliente": range(1, n_clientes + 1),
    "nome_cliente": [f"Cliente {i}" for i in range(1, n_clientes + 1)],
    "segmento_cliente": np.random.choice(["Retail", "B2B", "Parceiro"], size=n_clientes),
    "cidade": np.random.choice(["Porto", "Lisboa", "Braga", "Coimbra"], size=n_clientes),
    "pais": "Portugal"
})

# -------------------------
# DIM_ARTIGO
# -------------------------
n_artigos = 30
dim_artigo = pd.DataFrame({
    "id_artigo": range(1, n_artigos + 1),
    "nome_artigo": [f"Artigo {i}" for i in range(1, n_artigos + 1)],
    "categoria": np.random.choice(["Software", "Serviço", "Hardware"], size=n_artigos),
    "subcategoria": np.random.choice(["Licença", "Implementação", "Suporte", "Equipamento"], size=n_artigos)
})

# -------------------------
# DIM_VENDEDOR
# -------------------------
n_vendedores = 8
dim_vendedor = pd.DataFrame({
    "id_vendedor": range(1, n_vendedores + 1),
    "nome_vendedor": [f"Vendedor {i}" for i in range(1, n_vendedores + 1)],
    "equipa": np.random.choice(["Norte", "Sul", "Online"], size=n_vendedores)
})

# -------------------------
# DIM_TEMPO
# -------------------------
datas = pd.date_range(start="2024-01-01", end="2024-12-31", freq="D")
dim_tempo = pd.DataFrame({
    "id_tempo": range(1, len(datas) + 1),
    "data": datas,
})
dim_tempo["ano"] = dim_tempo["data"].dt.year
dim_tempo["mes"] = dim_tempo["data"].dt.month
dim_tempo["trimestre"] = dim_tempo["data"].dt.quarter
dim_tempo["dia_semana"] = dim_tempo["data"].dt.day_name(locale="pt_PT")

# -------------------------
# FACT_VENDA
# -------------------------
n_vendas = 3000  # nº de registos de vendas mock

fact_venda = pd.DataFrame({
    "id_venda": range(1, n_vendas + 1),
    "id_tempo": np.random.choice(dim_tempo["id_tempo"], size=n_vendas),
    "id_cliente": np.random.choice(dim_cliente["id_cliente"], size=n_vendas),
    "id_artigo": np.random.choice(dim_artigo["id_artigo"], size=n_vendas),
    "id_vendedor": np.random.choice(dim_vendedor["id_vendedor"], size=n_vendas),
    "quantidade": np.random.randint(1, 10, size=n_vendas),
    "preco_unitario": np.random.uniform(50, 1000, size=n_vendas).round(2),
    "desconto_perc": np.random.choice([0, 5, 10, 15, 20], size=n_vendas)
})

# calcula totais
fact_venda["custo_unitario"] = (fact_venda["preco_unitario"] * np.random.uniform(0.4, 0.8, size=n_vendas)).round(2)
fact_venda["total_bruto"] = (fact_venda["quantidade"] * fact_venda["preco_unitario"]).round(2)
fact_venda["total_desconto"] = (fact_venda["total_bruto"] * fact_venda["desconto_perc"] / 100).round(2)
fact_venda["total_liquido"] = (fact_venda["total_bruto"] - fact_venda["total_desconto"]).round(2)
fact_venda["custo_total"] = (fact_venda["quantidade"] * fact_venda["custo_unitario"]).round(2)
fact_venda["margem"] = (fact_venda["total_liquido"] - fact_venda["custo_total"]).round(2)

# quick check
dim_cliente.head(), dim_artigo.head(), dim_vendedor.head(), dim_tempo.head(), fact_venda.head()

In [ ]:
# ==============================
# Faturação por mês (para gráfico)
# ==============================

# agregação: soma da faturação líquida por ano + mês
faturacao_mes = (
    fact_join
    .groupby(["ano", "mes"], as_index=False)["total_liquido"]
    .sum()
    .sort_values(["ano", "mes"])
)

# coluna de conveniência para eixo X nos gráficos
faturacao_mes["ano_mes"] = (
    faturacao_mes["ano"].astype(str)
    + "-"
    + faturacao_mes["mes"].astype(str).str.zfill(2)
)

faturacao_mes.head()

In [ ]:
# ==============================
# Faturação por segmento de cliente
# ==============================

faturacao_segmento = (
    fact_join
    .groupby("segmento_cliente", as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
)

faturacao_segmento

In [ ]:
# ==============================
# Faturação por categoria de artigo
# ==============================

faturacao_categoria = (
    fact_join
    .groupby("categoria", as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
)

faturacao_categoria

In [ ]:
# ==============================
# Faturação por categoria de artigo
# ==============================

faturacao_categoria = (
    fact_join
    .groupby("categoria", as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
)

faturacao_categoria

In [ ]:
# ==============================
# Faturação por equipa de vendedor
# ==============================

faturacao_equipa = (
    fact_join
    .groupby("equipa", as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
)

faturacao_equipa

In [ ]:
# ==============================
# Top 10 Clientes por Faturação
# ==============================

top_clientes = (
    fact_join
    .groupby(["id_cliente", "nome_cliente"], as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
    .head(10)
)

top_clientes

In [ ]:
# ==============================
# Top 10 Artigos por Faturação
# ==============================

top_artigos = (
    fact_join
    .groupby(["id_artigo", "nome_artigo"], as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
    .head(10)
)

top_artigos

In [ ]:
# ==============================
# Top 10 Artigos por Margem
# ==============================

top_artigos_margem = (
    fact_join
    .groupby(["id_artigo", "nome_artigo"], as_index=False)["margem"]
    .sum()
    .sort_values("margem", ascending=False)
    .head(10)
)

top_artigos_margem

In [ ]:
# ==============================
# Tabela detalhe para dashboard
# ==============================

colunas_detalhe = [
    "id_venda",
    "data",
    "ano",
    "mes",
    "trimestre",
    "id_cliente",
    "nome_cliente",
    "segmento_cliente",
    "cidade",
    "id_artigo",
    "nome_artigo",
    "categoria",
    "subcategoria",
    "id_vendedor",
    "nome_vendedor",
    "equipa",
    "quantidade",
    "preco_unitario",
    "desconto_perc",
    "total_bruto",
    "total_desconto",
    "total_liquido",
    "custo_total",
    "margem",
]

tabela_detalhe = fact_join[colunas_detalhe].copy()

tabela_detalhe.head()

In [ ]:
tabela_detalhe.to_csv("tabela_detalhe_vendas.csv", index=False)

In [ ]:
# ==============================
# Faturação Mensal (últimos 24 meses)
# ==============================

faturacao_mensal = (
    fact_join
    .groupby(["ano", "mes"], as_index=False)["total_liquido"]
    .sum()
    .sort_values(["ano", "mes"])
)

# Criar coluna de período YYYY-MM
faturacao_mensal["periodo"] = faturacao_mensal["ano"].astype(str) + "-" + faturacao_mensal["mes"].astype(str).str.zfill(2)

faturacao_mensal

In [ ]:
# ==============================
# Gráfico - Faturação Mensal
# ==============================

import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))

plt.plot(
    faturacao_mensal["periodo"],
    faturacao_mensal["total_liquido"],
    marker="o",
    linewidth=2
)

plt.title("Faturação Mensal", fontsize=16)
plt.xlabel("Período (YYYY-MM)", fontsize=12)
plt.ylabel("Faturação (€)", fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# ==============================
# Gráfico - Faturação por Segmento de Cliente
# ==============================

import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))

plt.bar(
    faturacao_segmento["segmento_cliente"],
    faturacao_segmento["total_liquido"],
    color=["#1f77b4", "#ff7f0e", "#2ca02c"]  # cores diferentes para cada segmento
)

plt.title("Faturação por Segmento de Cliente", fontsize=16)
plt.xlabel("Segmento", fontsize=12)
plt.ylabel("Faturação (€)", fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# ==============================
# Margem Mensal (valor e %)
# ==============================

margem_mensal = (
    fact_join
    .groupby(["ano", "mes"], as_index=False)[["total_liquido", "margem"]]
    .sum()
    .sort_values(["ano", "mes"])
)

# Coluna período YYYY-MM (igual à faturação_mensal)
margem_mensal["periodo"] = (
    margem_mensal["ano"].astype(str)
    + "-"
    + margem_mensal["mes"].astype(str).str.zfill(2)
)

# Margem em percentagem por mês
margem_mensal["margem_pct"] = (
    margem_mensal["margem"] / margem_mensal["total_liquido"] * 100
).round(2)

margem_mensal

In [ ]:
# ==============================
# Gráfico – Margem Mensal (€)
# ==============================

plt.figure(figsize=(12,6))

plt.plot(
    margem_mensal["periodo"],
    margem_mensal["margem"],
    marker="o",
    linewidth=2
)

plt.title("Margem Mensal (€)", fontsize=16)
plt.xlabel("Período (YYYY-MM)", fontsize=12)
plt.ylabel("Margem (€)", fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# ==============================
# Gráfico – Margem Percentual (%)
# ==============================

plt.figure(figsize=(12,6))

plt.plot(
    margem_mensal["periodo"],
    margem_mensal["margem_pct"],
    marker="o",
    color="green",
    linewidth=2
)

plt.title("Margem Percentual Mensal (%)", fontsize=16)
plt.xlabel("Período (YYYY-MM)", fontsize=12)
plt.ylabel("Margem (%)", fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# ==============================
# Clientes Inativos (>180 dias)
# ==============================

# Última data de venda por cliente
ultima_venda = (
    fact_join.groupby("id_cliente")["data"]
    .max()
    .reset_index()
    .rename(columns={"data": "ultima_data"})
)

# Junta nomes dos clientes
ultima_venda = ultima_venda.merge(
    dim_cliente[["id_cliente", "nome_cliente"]],
    on="id_cliente",
    how="left"
)

# Define data de referência = última data total na base
data_referencia = fact_join["data"].max()

# Calcula inatividade em dias
ultima_venda["dias_inativo"] = (data_referencia - ultima_venda["ultima_data"]).dt.days

# Filtra clientes inativos (> 180 dias)
clientes_inativos = ultima_venda[ultima_venda["dias_inativo"] > 180] \
    .sort_values("dias_inativo", ascending=False)

clientes_inativos

In [ ]:
# =====================================
# Concentração de Faturação (Pareto 80/20)
# =====================================

pareto = (
    fact_join.groupby(["id_cliente", "nome_cliente"], as_index=False)["total_liquido"]
    .sum()
    .rename(columns={"total_liquido": "faturacao_total"})
    .sort_values("faturacao_total", ascending=False)
)

# Faturação total global
total_global = pareto["faturacao_total"].sum()

# Percentagem individual
pareto["pct"] = pareto["faturacao_total"] / total_global * 100

# Percentagem acumulada
pareto["pct_acumulado"] = pareto["pct"].cumsum().round(2)

pareto

In [ ]:
# =====================================
# Concentração de Faturação (Pareto 80/20)
# =====================================

pareto = (
    fact_join.groupby(["id_cliente", "nome_cliente"], as_index=False)["total_liquido"]
    .sum()
    .rename(columns={"total_liquido": "faturacao_total"})
    .sort_values("faturacao_total", ascending=False)
)

# Faturação total global
total_global = pareto["faturacao_total"].sum()

# Percentagem individual
pareto["pct"] = pareto["faturacao_total"] / total_global * 100

# Percentagem acumulada
pareto["pct_acumulado"] = pareto["pct"].cumsum().round(2)

pareto

In [ ]:
# 📘 Plano Técnico — Sistema de KPIs e Analytics PHC

## 1. Objetivo do Projeto
Criar uma estrutura analítica baseada nos dados extraídos do PHC, capaz de gerar KPIs comerciais, financeiros e operacionais, organizados em dashboards reutilizáveis.

---

## 2. Modelo de Dados (Mock PHC)
Foi criado um modelo estrela para simular a base de dados PHC:

### **Dimensões**
- `dim_cliente`: id_cliente, nome_cliente, segmento, cidade, país  
- `dim_artigo`: id_artigo, nome_artigo, categoria, subcategoria  
- `dim_vendedor`: id_vendedor, nome_vendedor, equipa  
- `dim_tempo`: data, ano, mês, trimestre, dia_semana  

### **Tabela Fato**
- `fact_venda`: vendas detalhadas com quantidade, preço, custo, total_liquido, margem, mês/ano e chaves das dimensões.

### **Tabelas exportadas (CSV)**
- `dim_cliente.csv`  
- `dim_artigo.csv`  
- `dim_vendedor.csv`  
- `fact_venda.csv`  
- `tabela_detalhe_vendas.csv`

Estas podem ser carregadas para PostgreSQL, SQLite, Power BI ou qualquer BI.

---

## 3. KPIs Implementados

### **Visão Geral**
- Faturação total  
- Margem total (€)  
- Margem percentual (%)  
- Nº de vendas  
- Nº clientes  
- Nº artigos  
- Ticket médio  
- Faturação média por cliente  
- Tabela `kpi_cards`

### **KPIs Temporais (mensal)**
- `faturacao_mensal`  
- `margem_mensal`  
- `margem_pct` (margem percentual por mês)

### **Clientes**
- `top_clientes` — Top 10 por faturação  
- `clientes_inativos` — Sem compras > 180 dias  
- Faturação por segmento (Retail / B2B / Parceiro)

### **Produtos / Artigos**
- `top_artigos` — Top 10 por faturação  
- `top_artigos_margem` — Top 10 por margem  
- Faturação por categoria  
- Faturação por equipa de vendedor

---

## 4. Dashboards a criar (Power BI / Metabase / Superset)

### **Dashboard 1 — Visão Geral**
- Faturação mensal (linha)
- Margem mensal (€ e %)
- Ticket médio
- KPIs principais (cards)

### **Dashboard 2 — Clientes**
- Top clientes
- Clientes inativos
- Faturação por segmento
- Concentração (Pareto 80/20)

### **Dashboard 3 — Produtos / Serviços**
- Top artigos por faturação
- Top artigos por margem
- Faturação por categoria

### **Filtros globais**
- Ano
- Mês
- Segmento
- Categoria

---

## 5. Próximos Passos (com BD real PHC)

### **Quando a documentação do PHC estiver disponível:**
1. Ler estrutura das tabelas reais.  
2. Ajustar modelo estrela ao PHC real.  
3. Criar scripts de extração (Python + SQLAlchemy ou SQL puro).  
4. Normalizar campos (datas, nomes, duplicados).  
5. Recalcular todos os KPIs diretamente sobre dados reais.  
6. Publicar dashboards com refresh automático.

---

## 6. Entregáveis Prontos (fase mock)
- Modelo de dados
- KPIs calculados
- Tabelas preparadas para BI
- Gráficos exploratórios
- Plano Técnico (este documento)
- Ficheiros CSV exportados

# 📘 Plano Técnico — Sistema de KPIs e Analytics PHC

## 1. Objetivo do Projeto
Criar uma estrutura analítica baseada nos dados extraídos do PHC, capaz de gerar KPIs comerciais, financeiros e operacionais, organizados em dashboards reutilizáveis.

---

## 2. Modelo de Dados (Mock PHC)
Foi criado um modelo estrela para simular a base de dados PHC:

### **Dimensões**
- `dim_cliente`: id_cliente, nome_cliente, segmento, cidade, país  
- `dim_artigo`: id_artigo, nome_artigo, categoria, subcategoria  
- `dim_vendedor`: id_vendedor, nome_vendedor, equipa  
- `dim_tempo`: data, ano, mês, trimestre, dia_semana  

### **Tabela Fato**
- `fact_venda`: vendas detalhadas com quantidade, preço, custo, total_liquido, margem, mês/ano e chaves das dimensões.

### **Tabelas exportadas (CSV)**
- `dim_cliente.csv`  
- `dim_artigo.csv`  
- `dim_vendedor.csv`  
- `fact_venda.csv`  
- `tabela_detalhe_vendas.csv`

Estas podem ser carregadas para PostgreSQL, SQLite, Power BI ou qualquer BI.

---

## 3. KPIs Implementados

### **Visão Geral**
- Faturação total  
- Margem total (€)  
- Margem percentual (%)  
- Nº de vendas  
- Nº clientes  
- Nº artigos  
- Ticket médio  
- Faturação média por cliente  
- Tabela `kpi_cards`

### **KPIs Temporais (mensal)**
- `faturacao_mensal`  
- `margem_mensal`  
- `margem_pct` (margem percentual por mês)

### **Clientes**
- `top_clientes` — Top 10 por faturação  
- `clientes_inativos` — Sem compras > 180 dias  
- Faturação por segmento (Retail / B2B / Parceiro)

### **Produtos / Artigos**
- `top_artigos` — Top 10 por faturação  
- `top_artigos_margem` — Top 10 por margem  
- Faturação por categoria  
- Faturação por equipa de vendedor

---

## 4. Dashboards a criar (Power BI / Metabase / Superset)

### **Dashboard 1 — Visão Geral**
- Faturação mensal (linha)
- Margem mensal (€ e %)
- Ticket médio
- KPIs principais (cards)

### **Dashboard 2 — Clientes**
- Top clientes
- Clientes inativos
- Faturação por segmento
- Concentração (Pareto 80/20)

### **Dashboard 3 — Produtos / Serviços**
- Top artigos por faturação
- Top artigos por margem
- Faturação por categoria

### **Filtros globais**
- Ano
- Mês
- Segmento
- Categoria

---

## 5. Próximos Passos (com BD real PHC)

### **Quando a documentação do PHC estiver disponível:**
1. Ler estrutura das tabelas reais.  
2. Ajustar modelo estrela ao PHC real.  
3. Criar scripts de extração (Python + SQLAlchemy ou SQL puro).  
4. Normalizar campos (datas, nomes, duplicados).  
5. Recalcular todos os KPIs diretamente sobre dados reais.  
6. Publicar dashboards com refresh automático.

---

## 6. Entregáveis Prontos (fase mock)
- Modelo de dados
- KPIs calculados
- Tabelas preparadas para BI
- Gráficos exploratórios
- Plano Técnico (este documento)
- Ficheiros CSV exportados

In [ ]:
# ==============================
# KPI Cards - Visão Geral
# ==============================

# KPIs base
faturacao_total = fact_join["total_liquido"].sum()
margem_total = fact_join["margem"].sum()
n_vendas = fact_join["id_venda"].nunique()
n_clientes = dim_cliente["id_cliente"].nunique()
n_artigos = dim_artigo["id_artigo"].nunique()

ticket_medio = faturacao_total / n_vendas
faturacao_media_por_cliente = faturacao_total / n_clientes
margem_pct = (margem_total / faturacao_total) * 100

# Tabela de KPI cards
kpi_cards = pd.DataFrame({
    "KPI": [
        "Faturação total (€)",
        "Margem total (€)",
        "Margem (%)",
        "Nº vendas",
        "Nº clientes",
        "Nº artigos",
        "Ticket médio (€)",
        "Faturação média por cliente (€)",
    ],
    "Valor": [
        round(faturacao_total, 2),
        round(margem_total, 2),
        round(margem_pct, 2),
        int(n_vendas),
        int(n_clientes),
        int(n_artigos),
        round(ticket_medio, 2),
        round(faturacao_media_por_cliente, 2),
    ],
})

kpi_cards

## 📊 Dashboard 1 — Visão Geral

Este dashboard resume o desempenho global da empresa, com base nos dados do PHC (mock):

- Faturação total, margem, nº vendas, nº clientes, nº artigos, ticket médio, faturação média por cliente  
  → Fonte: tabela `kpi_cards`

- Evolução da **faturação mensal (€)**  
  → Fonte: tabela `faturacao_mensal`

- Evolução da **margem mensal (€)**  
  → Fonte: tabela `margem_mensal["margem"]`

- Evolução da **margem percentual mensal (%)**  
  → Fonte: tabela `margem_mensal["margem_pct"]`

Este dashboard responde essencialmente a:
- “Quanto vendemos por mês?”
- “Qual é a margem em valor e em percentagem?”
- “O negócio está estável ou a crescer ao longo do ano?”

In [ ]:
# ==============================
# Dashboard 1 — Visão Geral (gráficos)
# Usa os dataframes já criados:
#   - faturacao_mensal
#   - margem_mensal
# ==============================

import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 1, figsize=(14, 14), sharex=True)

# --- Gráfico 1: Faturação Mensal (€)
axes[0].plot(
    faturacao_mensal["periodo"],
    faturacao_mensal["total_liquido"],
    marker="o",
    linewidth=2,
)
axes[0].set_title("Faturação Mensal (€)", fontsize=14)
axes[0].set_ylabel("Faturação (€)", fontsize=12)
axes[0].grid(True, linestyle="--", alpha=0.5)

# --- Gráfico 2: Margem Mensal (€)
axes[1].plot(
    margem_mensal["periodo"],
    margem_mensal["margem"],
    marker="o",
    linewidth=2,
)
axes[1].set_title("Margem Mensal (€)", fontsize=14)
axes[1].set_ylabel("Margem (€)", fontsize=12)
axes[1].grid(True, linestyle="--", alpha=0.5)

# --- Gráfico 3: Margem Percentual Mensal (%)
axes[2].plot(
    margem_mensal["periodo"],
    margem_mensal["margem_pct"],
    marker="o",
    linewidth=2,
)
axes[2].set_title("Margem Percentual Mensal (%)", fontsize=14)
axes[2].set_xlabel("Período (YYYY-MM)", fontsize=12)
axes[2].set_ylabel("Margem (%)", fontsize=12)
axes[2].grid(True, linestyle="--", alpha=0.5)
axes[2].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

## 📊 Dashboard 2 — Clientes

Este dashboard analisa o comportamento dos clientes, com foco em:
- Top clientes por faturação;
- Clientes inativos (sem compras há mais de 180 dias);
- Concentração de faturação (regra de Pareto 80/20);
- Segmentação por tipo de cliente (B2B, Retail, Parceiro).

Os dados são calculados a partir da tabela `fact_join` (vendas) ligada à dimensão `dim_cliente`.

In [ ]:
# ==============================
# Top 10 Clientes por Faturação
# ==============================

# Tabela TOP 10 (id_cliente, nome_cliente, total_liquido)
top_clientes = (
    fact_join
    .groupby(["id_cliente", "nome_cliente"], as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
    .head(10)
)

top_clientes

In [ ]:
# Gráfico de barras — Top 10 Clientes

plt.figure(figsize=(10, 5))

plt.bar(
    top_clientes["nome_cliente"],
    top_clientes["total_liquido"]
)

plt.title("Top 10 Clientes por Faturação", fontsize=14)
plt.xlabel("Cliente", fontsize=12)
plt.ylabel("Faturação (€)", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# ==============================
# Clientes Inativos (> 180 dias)
# ==============================

# Última data de venda por cliente
ultima_venda = (
    fact_join
    .groupby("id_cliente")["data"]
    .max()
    .reset_index()
    .rename(columns={"data": "ultima_data"})
)

# Junta nome do cliente
ultima_venda = ultima_venda.merge(
    dim_cliente[["id_cliente", "nome_cliente"]],
    on="id_cliente",
    how="left"
)

# Data de referência = data mais recente da base
data_referencia = fact_join["data"].max()

# Quantos dias desde a última compra
ultima_venda["dias_inativo"] = (data_referencia - ultima_venda["ultima_data"]).dt.days

# Filtra clientes com mais de 180 dias sem compras
clientes_inativos = (
    ultima_venda[ultima_venda["dias_inativo"] > 180]
    .sort_values("dias_inativo", ascending=False)
)

clientes_inativos.head(20)

In [ ]:
# ==============================
# Concentração de Faturação (Pareto 80/20)
# ==============================

# Faturação por cliente
faturacao_cliente = (
    fact_join
    .groupby(["id_cliente", "nome_cliente"], as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
)

# Total global
total_global = faturacao_cliente["total_liquido"].sum()

# Percentagem e percentagem acumulada
faturacao_cliente["pct"] = faturacao_cliente["total_liquido"] / total_global * 100
faturacao_cliente["pct_acum"] = faturacao_cliente["pct"].cumsum()

# Quantos clientes fazem até 80% da faturação?
clientes_80 = faturacao_cliente[faturacao_cliente["pct_acum"] <= 80]
n_clientes_80 = len(clientes_80)
pct_clientes_80 = n_clientes_80 / faturacao_cliente.shape[0] * 100

print(f"Clientes necessários para chegar a 80% da faturação: {n_clientes_80}")
print(f"Isto representa {pct_clientes_80:.1f}% da base total de clientes.")

In [ ]:
# Gráfico de Pareto (barras + linha de % acumulada)

fig, ax1 = plt.subplots(figsize=(10, 5))

# Barras: faturação por cliente (eixo esquerdo)
ax1.bar(
    faturacao_cliente["nome_cliente"],
    faturacao_cliente["total_liquido"]
)
ax1.set_xlabel("Cliente", fontsize=12)
ax1.set_ylabel("Faturação (€)", fontsize=12)
ax1.tick_params(axis="x", rotation=45, labelrotation=45)
ax1.grid(axis="y", linestyle="--", alpha=0.5)

# Linha: percentagem acumulada (eixo direito)
ax2 = ax1.twinx()
ax2.plot(
    faturacao_cliente["nome_cliente"],
    faturacao_cliente["pct_acum"],
    marker="o",
    linewidth=2
)
ax2.set_ylabel("% Faturação acumulada", fontsize=12)
ax2.set_ylim(0, 100)

plt.title("Concentração de Faturação por Cliente (Curva de Pareto)", fontsize=14)
fig.tight_layout()
plt.show()

In [ ]:
# ==============================
# Faturação por Cliente (Ranking Completo)
# ==============================

faturacao_cliente = (
    fact_join
    .groupby(["id_cliente", "nome_cliente"], as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
)

faturacao_cliente.head(10)  # mostra o top 10

In [ ]:
# Ranking correto de faturação por cliente
faturacao_cliente = (
    fact_join
    .groupby(["id_cliente", "nome_cliente"], as_index=False)
    .agg({"total_liquido": "sum"})
    .sort_values("total_liquido", ascending=False)
)

faturacao_cliente.head(10)

In [ ]:
faturacao_cliente = faturacao_cliente.reset_index(drop=True)
faturacao_cliente.index += 1  # começa em 1
faturacao_cliente.head(10)

In [ ]:
# ==========================================================
# Pareto 80/20 - Clientes
# ==========================================================

pareto = (
    faturacao_cliente
    .sort_values("total_liquido", ascending=False)
    .assign(pct=lambda x: x["total_liquido"] / x["total_liquido"].sum())
    .assign(pct_acum=lambda x: x["pct"].cumsum())
)

pareto.head(15)

In [ ]:
# Número de clientes necessários para atingir 80% da receita
clientes_80 = pareto.query("pct_acum <= 0.80").shape[0]
clientes_80

In [ ]:
# ==========================================================
# Segmentação ABC
# ==========================================================

def classificar_abc(p):
    if p <= 0.80:
        return "A"     # clientes críticos
    elif p <= 0.95:
        return "B"     # importantes mas não críticos
    else:
        return "C"     # baixo impacto

pareto["segmento"] = pareto["pct_acum"].apply(classificar_abc)

pareto.head(15)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

plt.plot(pareto.index, pareto["pct_acum"], marker="o")
plt.axhline(0.80, color="red", linestyle="--", label="80%")
plt.title("Curva Pareto - Clientes")
plt.xlabel("Clientes (ordenados por faturação)")
plt.ylabel("Percentagem acumulada")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# =========================================
# Gráfico: Top 10 Clientes por Faturação
# =========================================

top10 = faturacao_cliente.head(10)

plt.figure(figsize=(10,5))
plt.barh(top10["nome_cliente"], top10["total_liquido"], color="steelblue")
plt.gca().invert_yaxis()
plt.title("Top 10 Clientes por Faturação (€)")
plt.xlabel("Faturação (€)")
plt.ylabel("Cliente")
plt.grid(axis="x", linestyle="--", alpha=0.5)
plt.show()

In [ ]:
# ================================
# Gráfico: Clientes Inativos (>180 dias)
# ================================

clientes_inativos_plot = clientes_inativos.copy()
clientes_inativos_plot["nome"] = clientes_inativos_plot["nome_cliente"]

plt.figure(figsize=(10,5))
plt.barh(clientes_inativos_plot["nome"], clientes_inativos_plot["dias_inativo"], color="salmon")
plt.gca().invert_yaxis()

plt.title("Clientes Inativos (+180 dias)")
plt.xlabel("Dias sem compras")
plt.ylabel("Cliente")
plt.grid(axis="x", linestyle="--", alpha=0.5)

plt.show()

In [ ]:
# ================================
# Distribuição por Segmento (ABC) - contagem
# ================================
pareto["segmento"].value_counts()

In [ ]:
# ================================
# Gráfico: Faturação por Segmento (ABC)
# ================================

fat_segmento = (
    pareto.groupby("segmento", as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
)

plt.figure(figsize=(8,4))
plt.bar(fat_segmento["segmento"], fat_segmento["total_liquido"])
plt.title("Faturação por Segmento (ABC)")
plt.xlabel("Segmento")
plt.ylabel("Total Líquido (€)")
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

fat_segmento

In [ ]:
# =========================================
# Dashboard 3 — Produtos / Serviços
# =========================================

import matplotlib.pyplot as plt

# 1) Top 10 Artigos por Faturação
fat_artigo = (
    fact_join.groupby(["id_artigo", "nome_artigo"], as_index=False)["total_liquido"]
    .sum()
    .sort_values("total_liquido", ascending=False)
)

top10_artigos_fat = fat_artigo.head(10)

plt.figure(figsize=(10,5))
plt.barh(top10_artigos_fat["nome_artigo"], top10_artigos_fat["total_liquido"])
plt.gca().invert_yaxis()
plt.title("Top 10 Artigos por Faturação (€)")
plt.xlabel("Faturação (€)")
plt.ylabel("Artigo")
plt.grid(axis="x", linestyle="--", alpha=0.5)
plt.show()

top10_artigos_fat


# 2) Top 10 Artigos por Margem (€)
marg_artigo = (
    fact_join.groupby(["id_artigo", "nome_artigo"], as_index=False)["margem"]
    .sum()
    .sort_values("margem", ascending=False)
)

top10_artigos_marg = marg_artigo.head(10)

plt.figure(figsize=(10,5))
plt.barh(top10_artigos_marg["nome_artigo"], top10_artigos_marg["margem"])
plt.gca().invert_yaxis()
plt.title("Top 10 Artigos por Margem (€)")
plt.xlabel("Margem (€)")
plt.ylabel("Artigo")
plt.grid(axis="x", linestyle="--", alpha=0.5)
plt.show()

top10_artigos_marg


# 3) Faturação por Categoria (se existir coluna categoria em fact_join)
if "categoria" in fact_join.columns:
    fat_categoria = (
        fact_join.groupby("categoria", as_index=False)["total_liquido"]
        .sum()
        .sort_values("total_liquido", ascending=False)
    )

    plt.figure(figsize=(8,4))
    plt.bar(fat_categoria["categoria"], fat_categoria["total_liquido"])
    plt.title("Faturação por Categoria (€)")
    plt.xlabel("Categoria")
    plt.ylabel("Faturação (€)")
    plt.grid(axis="y", linestyle="--", alpha=0.5)
    plt.show()

    fat_categoria
else:
    print("⚠️ A coluna 'categoria' não está em fact_join. Diz-me onde tens 'categoria' (dim_artigo?) que eu ajusto o merge.")

In [ ]:
from pathlib import Path

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# 1. KPI Cards
kpi_cards.to_csv(OUTPUT_DIR / "kpi_cards.csv", index=False)

# 2. Faturação mensal
faturacao_mensal.to_csv(OUTPUT_DIR / "faturacao_mensal.csv", index=False)

# 3. Margem mensal
margem_mensal.to_csv(OUTPUT_DIR / "margem_mensal.csv", index=False)

# 4. Top clientes
top_clientes.to_csv(OUTPUT_DIR / "top_clientes.csv", index=False)

# 5. Clientes inativos
clientes_inativos.to_csv(OUTPUT_DIR / "clientes_inativos.csv", index=False)

# 6. Top artigos por faturação
top_artigos.to_csv(OUTPUT_DIR / "top_artigos.csv", index=False)

# 7. Top artigos por margem
top_artigos_margem.to_csv(OUTPUT_DIR / "top_artigos_margem.csv", index=False)

print("✅ CSVs exportados com sucesso para /outputs")

In [ ]:
# Ver variáveis parecidas com "top_artigos"
[k for k in globals().keys() if "top" in k and "art" in k]

In [ ]:
from pathlib import Path

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

kpi_cards.to_csv(OUTPUT_DIR / "kpi_cards.csv", index=False)
faturacao_mensal.to_csv(OUTPUT_DIR / "faturacao_mensal.csv", index=False)
margem_mensal.to_csv(OUTPUT_DIR / "margem_mensal.csv", index=False)
top_clientes.to_csv(OUTPUT_DIR / "top_clientes.csv", index=False)
clientes_inativos.to_csv(OUTPUT_DIR / "clientes_inativos.csv", index=False)
top_artigos.to_csv(OUTPUT_DIR / "top_artigos.csv", index=False)
top_artigos_margem.to_csv(OUTPUT_DIR / "top_artigos_margem.csv", index=False)

print("CSVs exportados para /outputs")

In [ ]:
print("fact_join exists:", "fact_join" in globals())
if "fact_join" in globals():
    print("fact_join shape:", fact_join.shape)
    print("cols:", list(fact_join.columns)[:10])

In [ ]:
print("fact_venda:", "fact_venda" in globals())
print("dim_tempo:", "dim_tempo" in globals())
print("dim_cliente:", "dim_cliente" in globals())
print("dim_artigo:", "dim_artigo" in globals())
print("dim_vendedor:", "dim_vendedor" in globals())

In [ ]:
# Recriar fact_join agora (forçar criação)
fact_join = fact_venda.merge(
    dim_tempo[["id_tempo", "data", "ano", "mes", "trimestre"]],
    on="id_tempo",
    how="left",
)

fact_join = fact_join.merge(
    dim_cliente[["id_cliente", "nome_cliente", "segmento_cliente", "cidade", "pais"]],
    on="id_cliente",
    how="left",
)

fact_join = fact_join.merge(
    dim_artigo[["id_artigo", "nome_artigo", "categoria", "subcategoria"]],
    on="id_artigo",
    how="left",
)

fact_join = fact_join.merge(
    dim_vendedor[["id_vendedor", "nome_vendedor", "equipa"]],
    on="id_vendedor",
    how="left",
)

print("fact_join exists:", "fact_join" in globals())
print("fact_join shape:", fact_join.shape)
fact_join.head()

In [ ]:
"fact_join" in globals(), fact_join.shape

In [ ]:
ls -lah outputs/

In [ ]:
jupyter nbconvert --clear-output --inplace 01_mock_data_final.ipynb

In [ ]:
print("fact_venda:", "fact_venda" in globals())
print("dim_cliente:", "dim_cliente" in globals())
print("dim_artigo:", "dim_artigo" in globals())
print("dim_tempo:", "dim_tempo" in globals())
print("dim_vendedor:", "dim_vendedor" in globals())
print("fact_join:", "fact_join" in globals())

if "fact_join" in globals():
    print("fact_join shape:", fact_join.shape)
    print("KPIs OK:", "kpi_cards" in globals())